In [2]:
 # 1. Install dependencies
!pip install transformers
!pip install gradio # Gradio permits to build interactives web interfaces

import torch

from transformers import AutoModelForCausalLM, AutoTokenizer

modelo = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizador = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

chat_historial = []

def generar_respuesta(input_text, chat_historial, max_length=1000):
    # Tokenizar entrada junto con el historial
    nueva_entrada = tokenizador.encode(input_text + tokenizador.eos_token, return_tensors="pt")

    # Agregar al historial si no está vacío
    if len(chat_historial) > 0:
        nueva_entrada = chat_historial + nueva_entrada.tolist()[0]
    else:
        nueva_entrada = nueva_entrada[0].tolist()

    # Generar respuesta
    respuesta_ids = modelo.generate(
        input_ids=torch.tensor([nueva_entrada]),
        max_length=max_length,
        pad_token_id=tokenizador.eos_token_id,
        top_p=0.9,  # Controla qué tokens considerar en la respuesta
        temperature=0.8  # Controla la creatividad del modelo
    )

    # Decodificar la respuesta generada
    respuesta = tokenizador.decode(respuesta_ids[:, len(nueva_entrada):][0], skip_special_tokens=True)

    # Actualizar historial
    chat_historial.extend(respuesta_ids.tolist()[0])

    return respuesta, chat_historial

In [3]:
# Ciclo de conversación
chat_historial = []
print("Chatbot avanzado. Escribe 'salir' para terminar.\n")

while True:
    entrada = input("Tú: ")
    if entrada.lower() == "salir":
        break
    respuesta, chat_historial = generar_respuesta(entrada, chat_historial)
    print(f"Chatbot: {respuesta}")

Chatbot avanzado. Escribe 'salir' para terminar.

Tú: Good evening
Chatbot: Good morning
Tú: How are you?
Chatbot: I'm good, how are you?
Tú: salir


In [5]:
import gradio as gr
# Funcion para manejar interaccion
def chatbot_gradio(input_text, chat_historial=[]):
  respuesta, chat_historial =  generar_respuesta(input_text, chat_historial)
  return respuesta, chat_historial

# Create interface
interfaz = gr.Interface(
    fn=chatbot_gradio,
    inputs=["text","state"],
    outputs=["text","state"],
    title="Chatbot Avanzado con Memoria",
    description="This chatbot remembers the context of conversation and responds"
)

# Launch interface
interfaz.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3117526a38c6ea47c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
